In [3]:
import pandas as pd
import mailchimp_modul as mcm
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError

%load_ext autoreload
%autoreload 2

# Replace with your actual API key and client ID
api_key = '4af88cb8f553cbf45b6a5de8bcb88a5f'
server = 'us17'
campaign_id = '3a09ce9393'

# API endpoint
# url = f'https://api.createsend.com/api/v3.2/clients/{client_id}/campaigns.json'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
import mailchimp_marketing as MailchimpMarketing
from mailchimp_marketing.api_client import ApiClientError

try:
  client = MailchimpMarketing.Client()
  client.set_config({
    "api_key": "4af88cb8f553cbf45b6a5de8bcb88a5f",
    "server": "us17"
  })

  response = client.campaigns.get("b29c3671f0")
  print(response)
except ApiClientError as error:
  print("Error: {}".format(error.text))

{'id': 'b29c3671f0', 'web_id': 13568625, 'type': 'regular', 'create_time': '2025-08-08T02:43:19+00:00', 'archive_url': 'http://eepurl.com/jkR3R6', 'long_archive_url': 'https://mailchi.mp/96ebcb64c846/75-off-points-on-your-mediterranean-escape-13568625', 'status': 'sent', 'emails_sent': 8308, 'send_time': '2025-08-08T14:33:40+00:00', 'content_type': 'html', 'needs_block_refresh': False, 'resendable': True, 'recipients': {'list_id': '7f6e7d27ac', 'list_is_active': True, 'list_name': 'Karma Group', 'segment_text': '<p class="!margin--lv0 display--inline">Contacts that match <strong>all</strong> of the following conditions:</p><ol id="conditions" class="small-meta text-transform--none"><li class="margin--lv1 !margin-left-right--lv0">Matches a custom advanced segment</li></ol><span>For a total of <strong>8308</strong> emails sent.</span>', 'recipient_count': 8308, 'segment_opts': {'match': 'all'}}, 'settings': {'subject_line': 'Meet the Wonders of the Ocean Like Never Before🦈', 'preview_tex

In [ ]:
# import mailchimp_marketing as MailchimpMarketing
# from mailchimp_marketing.api_client import ApiClientError

# try:
#   client = MailchimpMarketing.Client()
#   client.set_config({
#     "api_key": "4af88cb8f553cbf45b6a5de8bcb88a5f",
#     "server": "us17"
#   })

#   response = client.searchCampaigns.search(search_value)
#   print(response)
#   # Check the type of the response object
#   print(f"\nType of response object: {type(response)}")
# except ApiClientError as error:
#   print("Error: {}".format(error.text))

# response


{'results': [{'campaign': {'id': 'b29c3671f0', 'web_id': 13568625, 'type': 'regular', 'create_time': '2025-08-08T02:43:19+00:00', 'archive_url': 'http://eepurl.com/jkR3R6', 'long_archive_url': 'https://mailchi.mp/96ebcb64c846/75-off-points-on-your-mediterranean-escape-13568625', 'status': 'sent', 'emails_sent': 8308, 'send_time': '2025-08-08T14:33:40+00:00', 'content_type': 'html', 'needs_block_refresh': False, 'resendable': True, 'recipients': {'list_id': '7f6e7d27ac', 'list_is_active': True, 'list_name': 'Karma Group', 'segment_text': '<p class="!margin--lv0 display--inline">Contacts that match <strong>all</strong> of the following conditions:</p><ol id="conditions" class="small-meta text-transform--none"><li class="margin--lv1 !margin-left-right--lv0">Matches a custom advanced segment</li></ol><span>For a total of <strong>8308</strong> emails sent.</span>', 'recipient_count': 8308, 'segment_opts': {'match': 'all'}}, 'settings': {'subject_line': 'Meet the Wonders of the Ocean Like Ne

{'results': [{'campaign': {'id': 'b29c3671f0',
    'web_id': 13568625,
    'type': 'regular',
    'create_time': '2025-08-08T02:43:19+00:00',
    'archive_url': 'http://eepurl.com/jkR3R6',
    'long_archive_url': 'https://mailchi.mp/96ebcb64c846/75-off-points-on-your-mediterranean-escape-13568625',
    'status': 'sent',
    'emails_sent': 8308,
    'send_time': '2025-08-08T14:33:40+00:00',
    'content_type': 'html',
    'needs_block_refresh': False,
    'resendable': True,
    'recipients': {'list_id': '7f6e7d27ac',
     'list_is_active': True,
     'list_name': 'Karma Group',
     'segment_text': '<p class="!margin--lv0 display--inline">Contacts that match <strong>all</strong> of the following conditions:</p><ol id="conditions" class="small-meta text-transform--none"><li class="margin--lv1 !margin-left-right--lv0">Matches a custom advanced segment</li></ol><span>For a total of <strong>8308</strong> emails sent.</span>',
     'recipient_count': 8308,
     'segment_opts': {'match': 'al

In [61]:

def getOpeners(api_key, server, campaign_id):
    import mailchimp_marketing as MailchimpMarketing
    from mailchimp_marketing.api_client import ApiClientError
    # setup client
    client = MailchimpMarketing.Client()
    client.set_config({
        "api_key": api_key,
        "server": server
    })

    emails_openers = []
    offset = 0
    count = 200  # Set a higher count for each request

    try:
        while True:
            response_openers = client.reports.get_campaign_open_details(
                campaign_id,
                count=count,
                offset=offset
            )
            
            # Extract email addresses from the current page of results
            members = response_openers.get("members", [])
            if not members:
                # If no members are returned, we have reached the end of the list
                break
            
            for member in members:
                emails_openers.append(member["email_address"])
                
            # Increment the offset for the next page
            offset += count

        # Create DataFrame from all collected email addresses
        df_openers = pd.DataFrame(emails_openers, columns=["email_address"])
        # display(df_openers)
        return df_openers

    except ApiClientError as error:
        print(f"Error: {error.text}")

In [ ]:
def getClickers(api_key, server, campaign_id):
    import mailchimp_marketing as MailchimpMarketing
    from mailchimp_marketing.api_client import ApiClientError

    # setup client
    client = MailchimpMarketing.Client()
    client.set_config({
        "api_key": api_key,
        "server": server
    })

    rows = []

    try:
        # 1) Get all links in the campaign
        links_resp = client.reports.get_campaign_click_details(campaign_id)
        links = links_resp.get("urls_clicked", [])

        # 2) Loop through each link and get clickers
        for link in links:
            link_id = link["id"]
            url = link["url"]
            # print(f"Processing link: {url} (ID: {link_id})")

            offset, count = 0, 1000
            while True:
                resp = client.reports.get_subscribers_info(
                    campaign_id,
                    link_id,
                    count=count,
                    offset=offset
                )
                members = resp.get("members", [])

                for m in members:
                    email = m.get("email_address")
                    if email:
                        rows.append({
                            "email_address": email,
                            "contact_id": m.get("contact_id"),
                            "clicks_for_this_link": m.get("clicks"),
                            "url": url,
                            "link_id": link_id
                        })

                if len(members) < count:
                    break
                offset += count

    except ApiClientError as error:
        print(f"Error: {error.text}")

    # 3) Create DataFrames
    df_clickers = pd.DataFrame(rows)
    # Add a check to see if the DataFrame is empty before proceeding
    if not df_clickers.empty:
        df_unique_emails = df_clickers[["email_address"]].drop_duplicates().sort_values("email_address").reset_index(drop=True)
        
        # print("\n--- All Clickers (sample) ---")
        # display(df_clickers)

        # print("\n--- Unique Emails ---")
        # display(df_unique_emails)

        return df_clickers
        # return df_unique_emails
    else:
        print("\nNo click data found for this campaign.")

In [49]:
def getCampaignDetails(api_key, server, search_value):
    import mailchimp_marketing as MailchimpMarketing
    from mailchimp_marketing.api_client import ApiClientError
    
    try:
        client = MailchimpMarketing.Client()
        client.set_config({
            "api_key": api_key,
            "server": server
        })

        response = client.searchCampaigns.search(search_value)
        print(response) # Keep this if you want to see the raw response

        data = response

        # Initialize lists to collect DataFrames
        extracted_campaign_data = []
        all_openers_dfs = []
        all_clickers_dfs = []
        all_unique_clickers_dfs = []
        all_summary_dfs = []

        for campaign_info in data['results']:
            campaign = campaign_info['campaign']
            campaign_id = campaign['id']
            campaign_title = campaign['settings']['title']

            # 1. Collect basic campaign info
            extracted_campaign_data.append({'id': campaign_id, 'title': campaign_title})

            # 2. Get Openers DataFrame
            current_openers_df = getOpeners(api_key, server, campaign_id)
            # Add campaign_name column to the current openers DataFrame
            current_openers_df['campaign_name'] = campaign_title
            all_openers_dfs.append(current_openers_df) # Add to the list

            # 3. Get Clickers DataFrame
            current_clickers_df = getClickers(api_key, server, campaign_id)
            # Add campaign_name column to the current clickers DataFrame
            current_clickers_df['campaign_name'] = campaign_title
            all_clickers_dfs.append(current_clickers_df) # Add to the list

            # 4. Calculate counts
            count_open = current_openers_df.shape[0] # Number of rows in openers

            # 5. Process unique clickers
            if not current_clickers_df.empty:
                # Correctly derive current_unique_clickers_df from current_clickers_df
                current_unique_clickers_df = current_clickers_df[["email_address"]].drop_duplicates().sort_values("email_address").reset_index(drop=True)
                current_unique_clickers_df['campaign_name'] = campaign_title
                all_unique_clickers_dfs.append(current_unique_clickers_df) # Add to the list
                count_click = current_clickers_df.shape[0] # Number of rows in clickers
            else:
                # If clickers are empty, unique clickers is also empty
                all_unique_clickers_dfs.append(pd.DataFrame(columns=['email_address', 'campaign_name'])) # Append empty DF with expected columns
                count_click = 0

            # 6. Create summary DataFrame for the current campaign
            # Use 'campaign_title' as the index for clarity or add it as a column
            # Changed index to column for easier concatenation later
            current_summary_df = pd.DataFrame({
                'campaign_title': [campaign_title],
                'open': [count_open],
                'click': [count_click]
            })
            all_summary_dfs.append(current_summary_df) # Add to the list

        # --- Concatenate all collected DataFrames outside the loop ---
        df_campaign = pd.DataFrame(extracted_campaign_data)

        # Concatenate only if lists are not empty to avoid errors on empty searches
        df_openers = pd.concat(all_openers_dfs, ignore_index=True) if all_openers_dfs else pd.DataFrame(columns=['email_address', 'open_time', 'campaign_name'])
        df_clickers = pd.concat(all_clickers_dfs, ignore_index=True) if all_clickers_dfs else pd.DataFrame(columns=['email_address', 'click_time', 'campaign_name'])
        df_unique_clickers = pd.concat(all_unique_clickers_dfs, ignore_index=True) if all_unique_clickers_dfs else pd.DataFrame(columns=['email_address', 'campaign_name'])
        df_summary = pd.concat(all_summary_dfs, ignore_index=True) if all_summary_dfs else pd.DataFrame(columns=['campaign_title', 'open', 'click'])

        # Display results
        display(df_campaign)

        print("\n--- Openers ---")
        display(df_openers)

        print("\n--- Clickers ---")
        display(df_clickers)

        print("\n--- Unique Clickers ---")
        display(df_unique_clickers)

        print("\n--- Summary ---")
        display(df_summary)

        return df_campaign # Or return all DataFrames if needed
    except ApiClientError as error:
        print(f"Error: {error.text}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

In [62]:
search_value = 'KCI_W1Aug25_Fushi_KRR'

# getCampaignDetails(api_key, server, search_value)
getOpeners(api_key, server, "b59db76d23")

,email_address
0,dbkknowles@gmail.com
1,elsa.natawid81@gmail.com
2,audreehadikrisno@gmail.com
3,michaella.prawatya@gmail.com
4,leo_taruna78@yahoo.co.id
...,...
456,princettanadja@gmail.com
457,rebeccakainama@gmail.com
458,eduard.chandra22@gmail.com
459,andreaschandr@gmail.com


In [63]:
getOpeners(api_key, server, '3a09ce9393')

,email_address
0,benny.risanto@karmagroup.com
1,dbkknowles@gmail.com
2,fajar.fatoni@karmagroup.com
3,emji.indra@gmail.com
4,drjeannecarpenter@icloud.com
...,...
455,benjamin7006@gmail.com
456,calloway72@hotmail.com
457,jzhy92@gmail.com
458,nicollevital14@gmail.com
